# Azure AI Agents Orchestration with Semantic Kernel - Complete Tutorial

🚀 **Master the Art of Multi-Agent Orchestration!**

This tutorial shows you how to orchestrate multiple **Azure AI Agents** working together using Semantic Kernel, both **sequentially** and **concurrently**!

## What You'll Learn:

1. **Sequential Orchestration** - Azure AI Agents working one after another in a pipeline
2. **Concurrent Orchestration** - Multiple Azure AI Agents working simultaneously
3. **Agent Specialization** - Creating specialized Azure AI Agents with unique plugins
4. **Workflow Patterns** - Real-world orchestration scenarios with Azure AI Agents
5. **Performance Optimization** - When to use concurrent vs sequential patterns

**Azure AI Configuration:**
- Uses **Azure AI Agents** (not ChatCompletionAgent)
- Uses **AzureChatCompletion** service (not OpenAIChatCompletion)
- Automatically configures Azure OpenAI connection using environment variables
- Compatible with Azure AI Studio deployments

**Prerequisites:** You should have completed the basic Azure AI Agents and Plugins tutorials first!

---

## 🎭 The Power of Agent Orchestration

Imagine having a team of **Azure AI Agents** working together on complex tasks!

### Single Agent Limitations:
- One agent handles everything
- Limited expertise in specific domains
- Potential context overload
- Slower processing for complex workflows

### Multi-Agent Orchestration Benefits:
- **Specialization**: Each Azure AI Agent excels in specific tasks 🎯
- **Parallel Processing**: Multiple Azure AI Agents work simultaneously ⚡
- **Better Results**: Domain experts produce higher quality outputs 📈
- **Scalability**: Add more Azure AI Agents as needed 🔄
- **Resilience**: If one agent fails, others continue 💪

### Orchestration Patterns:

- **Sequential (Pipeline)**: Azure AI Agent A → Azure AI Agent B → Azure AI Agent C
- **Concurrent (Parallel)**: Azure AI Agent A + Azure AI Agent B + Azure AI Agent C → Combine Results
- **Hybrid**: Mix of sequential and concurrent patterns

Let's build this step by step with Azure AI Agents! 🛠️

## Sequential Orchestration

![Sequential Orchestration](images/sequential_orchestration.gif)

## Concurrent Orchestration
![Concurrent Orchestration](images/concurrent_orchestration.gif)

## 📋 Setup and Prerequisites

Before we start orchestrating Azure AI Agents, let's make sure we have everything we need:

**Environment Variables Required:**
- `PROJECT_ENDPOINT`: Your Azure AI Project endpoint
- `MODEL_DEPLOYMENT_NAME`: Your deployed AI model name
- `AZURE_OPENAI_API_KEY`: Your Azure OpenAI API key (optional, can use DefaultAzureCredential)
- `AZURE_OPENAI_ENDPOINT`: Your Azure OpenAI endpoint (optional, can use DefaultAzureCredential)

**Azure Configuration:**
- Uses **Azure AI Agents** (not ChatCompletionAgent)
- Uses **AzureChatCompletion** service with DefaultAzureCredential
- Automatically configures Azure OpenAI connection using environment variables
- Compatible with Azure AI Studio deployments

We'll import all the necessary libraries and create our specialized plugins for each Azure AI Agent.

In [1]:
# Install required packages
# !pip install azure-ai-agents azure-identity semantic-kernel

# Import everything we need for Azure AI Agent orchestration
import os
import asyncio
import json
import time
import random
from datetime import datetime, timedelta
from typing import Any, Dict, List, Annotated

# Azure AI Agents and Semantic Kernel imports
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import (
    Agent,
    AzureAIAgent,
    ConcurrentOrchestration,
    SequentialOrchestration
)
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.functions import kernel_function
from semantic_kernel.contents import ChatMessageContent, AuthorRole
from semantic_kernel.agents.open_ai.run_polling_options import RunPollingOptions

print("🎭 Azure AI Agent orchestration packages imported successfully!")
print("🎯 Ready to create specialized Azure AI Agent teams with Semantic Kernel!")

# Verify environment variables for Azure AI Agents
required_vars = ['PROJECT_ENDPOINT', 'MODEL_DEPLOYMENT_NAME']
optional_vars = ['AZURE_OPENAI_ENDPOINT', 'AZURE_OPENAI_API_KEY']

for var in required_vars:
    if var in os.environ:
        print(f"✅ {var}: {'*' * (len(os.environ[var]) - 10)}{os.environ[var][-10:]}")
    else:
        print(f"❌ {var} is missing!")

for var in optional_vars:
    if var in os.environ:
        print(f"✅ {var}: Set (will use for fallback)")
    else:
        print(f"ℹ️ {var}: Not set (will use DefaultAzureCredential)")
        
print("\n🔧 Azure AI Agents will use DefaultAzureCredential for authentication!")
print("📘 Make sure you're signed into Azure CLI or have appropriate Azure credentials!")

🎭 Azure AI Agent orchestration packages imported successfully!
🎯 Ready to create specialized Azure AI Agent teams with Semantic Kernel!
✅ PROJECT_ENDPOINT: **********************************************************************************************************ry-project
✅ MODEL_DEPLOYMENT_NAME: gpt-4.1
✅ AZURE_OPENAI_ENDPOINT: Set (will use for fallback)
✅ AZURE_OPENAI_API_KEY: Set (will use for fallback)

🔧 Azure AI Agents will use DefaultAzureCredential for authentication!
📘 Make sure you're signed into Azure CLI or have appropriate Azure credentials!


## 🏗️ Creating Specialized Agent Plugins

Let's create three specialized Azure AI Agents with unique capabilities:

1. **🔍 Research Agent** - Information gathering and analysis
2. **📊 Analytics Agent** - Data processing and calculations
3. **📝 Content Agent** - Writing and formatting

Each Azure AI Agent will have specialized plugins that make them experts in their domain!

In [2]:
# 🔍 Research Agent Plugins
class ResearchPlugin:
    """Plugin for research and information gathering tasks."""
    
    @kernel_function(
        description="Researches information about a topic and provides key facts",
        name="research_topic"
    )
    def research_topic(
        self, 
        topic: Annotated[str, "The topic to research"]
    ) -> Annotated[str, "JSON string with research findings"]:
        """Simulates research on a topic."""
        # In a real implementation, this would call APIs, search databases, etc.
        research_data = {
            "topic": topic,
            "key_facts": [
                f"Key insight about {topic} - statistical trends",
                f"Recent developments in {topic} field",
                f"Expert opinions on {topic} future"
            ],
            "sources": ["Academic Papers", "Industry Reports", "Expert Interviews"],
            "confidence": 85,
            "research_time": datetime.now().isoformat()
        }
        print(f"🔍 Research completed for: {topic}")
        return json.dumps(research_data, indent=2)
    
    @kernel_function(
        description="Validates information sources and checks credibility",
        name="validate_sources"
    )
    def validate_sources(
        self,
        sources: Annotated[str, "List of sources to validate"]
    ) -> Annotated[str, "Validation results"]:
        """Validates information sources."""
        validation = {
            "validated_sources": sources.split(","),
            "credibility_score": random.randint(75, 95),
            "recommendations": "Sources appear credible and current"
        }
        print(f"🔍 Source validation completed")
        return json.dumps(validation)

# 📊 Analytics Agent Plugins
class AnalyticsPlugin:
    """Plugin for data analysis and calculations."""
    
    @kernel_function(
        description="Analyzes data and calculates statistics",
        name="analyze_data"
    )
    def analyze_data(
        self,
        data_description: Annotated[str, "Description of the data to analyze"]
    ) -> Annotated[str, "Analysis results with statistics"]:
        """Performs data analysis."""
        # Simulate complex data analysis
        analysis = {
            "data_analyzed": data_description,
            "metrics": {
                "mean": round(random.uniform(10, 100), 2),
                "median": round(random.uniform(10, 100), 2),
                "std_deviation": round(random.uniform(5, 20), 2),
                "sample_size": random.randint(100, 10000)
            },
            "trends": ["Upward trend", "Seasonal variation", "Growth acceleration"],
            "insights": f"Key patterns identified in {data_description}",
            "analysis_timestamp": datetime.now().isoformat()
        }
        print(f"📊 Data analysis completed for: {data_description}")
        return json.dumps(analysis, indent=2)
    
    @kernel_function(
        description="Creates projections and forecasts based on data",
        name="create_forecast"
    )
    def create_forecast(
        self,
        baseline_data: Annotated[str, "Baseline data for forecasting"]
    ) -> Annotated[str, "Forecast results"]:
        """Creates forecasts based on data."""
        forecast = {
            "forecast_period": "Next 6 months",
            "projected_values": [round(random.uniform(80, 120), 1) for _ in range(6)],
            "confidence_interval": "85-95%",
            "risk_factors": ["Market volatility", "Seasonal effects"],
            "recommendations": "Monitor key indicators weekly"
        }
        print(f"📊 Forecast created based on: {baseline_data}")
        return json.dumps(forecast)

# 📝 Content Agent Plugins
class ContentPlugin:
    """Plugin for content creation and formatting."""
    
    @kernel_function(
        description="Creates well-formatted reports from research and analysis data",
        name="create_report"
    )
    def create_report(
        self,
        research_data: Annotated[str, "Research findings to include"],
        analysis_data: Annotated[str, "Analysis results to include"]
    ) -> Annotated[str, "Formatted report"]:
        """Creates a formatted report from data."""
        report = f"""
# Executive Summary Report

**Generated on:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Research Findings
{research_data}

## Data Analysis
{analysis_data}

## Recommendations
- Implement data-driven strategies
- Monitor key performance indicators
- Schedule regular review cycles

---
*Report generated by AI Agent Orchestration System*
"""
        print(f"📝 Report created successfully")
        return report
    
    @kernel_function(
        description="Summarizes complex information into key points",
        name="create_summary"
    )
    def create_summary(
        self,
        content: Annotated[str, "Content to summarize"]
    ) -> Annotated[str, "Executive summary"]:
        """Creates an executive summary."""
        summary = {
            "executive_summary": "Key insights and recommendations from comprehensive analysis",
            "main_points": [
                "Research validates current market position",
                "Analytics reveal positive growth trends", 
                "Strategic opportunities identified"
            ],
            "action_items": [
                "Continue monitoring metrics",
                "Implement recommended changes",
                "Schedule follow-up analysis"
            ]
        }
        print(f"📝 Summary created")
        return json.dumps(summary, indent=2)

# Create plugin instances
research_plugin = ResearchPlugin()
analytics_plugin = AnalyticsPlugin()
content_plugin = ContentPlugin()

print("✅ All specialized plugins created!")
print("🔍 Research Plugin: Information gathering & validation")
print("📊 Analytics Plugin: Data analysis & forecasting")
print("📝 Content Plugin: Report creation & summarization")

✅ All specialized plugins created!
🔍 Research Plugin: Information gathering & validation
📊 Analytics Plugin: Data analysis & forecasting
📝 Content Plugin: Report creation & summarization


## 🤖 Creating Our Specialized Azure AI Agent Team

Now let's create three specialized Azure AI Agents, each with their own expertise and plugins:

1. **Dr. Research** 🔍 - The information specialist
2. **Prof. Analytics** 📊 - The data scientist  
3. **Ms. Content** 📝 - The communication expert

Each Azure AI Agent will have distinct instructions and specialized plugins to excel in their domain!

In [3]:
async def create_specialized_agents():
    """Create three specialized Azure AI Agents with unique capabilities using Semantic Kernel."""
    
    print("🤖 Creating specialized Azure AI Agent team...")
    
    endpoint = os.environ["PROJECT_ENDPOINT"]
    model_deployment = os.environ["MODEL_DEPLOYMENT_NAME"]
    agents = {}
    
    async with DefaultAzureCredential() as creds:
        client = AzureAIAgent.create_client(credential=creds, endpoint=endpoint)
        
        # 🔍 Create Research Agent
        print("🔍 Creating Research Agent...")
        research_agent_definition = await client.agents.create_agent(
            model=model_deployment,
            name="dr_research",
            description="World-class information specialist and fact-checker",
            instructions="""You are Dr. Research, a world-class information specialist and fact-checker.
            
            Your expertise:
            - Gathering comprehensive information on any topic
            - Validating sources and checking credibility
            - Providing well-researched, factual content
            - Identifying key insights and trends
            
            Always use your research plugins when asked to investigate topics.
            Be thorough, accurate, and cite your methodology.
            Focus on providing high-quality, verified information.
            Be VERY concise in your answers."""
        )
        
        research_agent = AzureAIAgent(
            client=client,
            definition=research_agent_definition,
            polling_options=RunPollingOptions(default_polling_interval=timedelta(milliseconds=3000)),
            plugins=[research_plugin]
        )
        agents['research'] = research_agent
        print(f"✅ Dr. Research: {research_agent_definition.id}")
        
        # 📊 Create Analytics Agent  
        print("📊 Creating Analytics Agent...")
        analytics_agent_definition = await client.agents.create_agent(
            model=model_deployment,
            name="prof_analytics",
            description="Brilliant data scientist and statistician",
            instructions="""You are Prof. Analytics, a brilliant data scientist and statistician.
            
            Your expertise:
            - Analyzing complex datasets and patterns
            - Creating statistical models and forecasts
            - Identifying trends and correlations
            - Providing data-driven insights and recommendations
            
            Always use your analytics plugins when working with data.
            Be precise with numbers and explain your analytical methodology.
            Focus on extracting actionable insights from information.
            Be VERY concise in your answers."""
        )
        
        analytics_agent = AzureAIAgent(
            client=client,
            definition=analytics_agent_definition,
            polling_options=RunPollingOptions(default_polling_interval=timedelta(milliseconds=3000)),
            plugins=[analytics_plugin]
        )
        agents['analytics'] = analytics_agent
        print(f"✅ Prof. Analytics: {analytics_agent_definition.id}")
        
        # 📝 Create Content Agent
        print("📝 Creating Content Agent...")
        content_agent_definition = await client.agents.create_agent(
            model=model_deployment,
            name="ms_content",
            description="Expert content creator and communication specialist",
            instructions="""You are Ms. Content, an expert content creator and communication specialist.
            
            Your expertise:
            - Creating clear, engaging, and well-structured content
            - Synthesizing complex information into readable formats
            - Writing reports, summaries, and presentations
            - Ensuring consistent tone and professional formatting
            
            Always use your content plugins when creating reports or summaries.
            Focus on clarity, readability, and professional presentation.
            Make complex information accessible to any audience.
            Be VERY concise in your answers."""
        )
        
        content_agent = AzureAIAgent(
            client=client,
            definition=content_agent_definition,
            polling_options=RunPollingOptions(default_polling_interval=timedelta(milliseconds=3000)),
            plugins=[content_plugin]
        )
        agents['content'] = content_agent
        print(f"✅ Ms. Content: {content_agent_definition.id}")
    
    print("🎭 Specialized Azure AI Agent team created!")
    print(f"🔍 Dr. Research: Information gathering specialist")
    print(f"📊 Prof. Analytics: Data analysis expert")
    print(f"📝 Ms. Content: Communication specialist")
    
    return agents

print("✅ Specialized Azure AI Agent creation function ready!")

✅ Specialized Azure AI Agent creation function ready!


## 🔄 Sequential Orchestration: Pipeline Pattern

In sequential orchestration, Azure AI Agents work one after another in a pipeline:

**Research Agent** → **Analytics Agent** → **Content Agent** → **Final Result**

This pattern is perfect when:
- Each Azure AI Agent depends on the previous agent's output
- You need a step-by-step workflow
- Quality control at each stage is important

Let's implement a sequential workflow for creating a comprehensive market analysis report!

In [4]:
def agent_response_callback(message: ChatMessageContent) -> None:
    """Callback function to monitor Azure AI Agent responses during orchestration."""
    if message.role != AuthorRole.TOOL:
        timestamp = datetime.now().strftime("%H:%M:%S")
        print(f"\n[{timestamp}] 🤖 Azure AI Agent: {message.name}")
        print(f"📝 Response: {message.content[:200]}..." if len(str(message.content)) > 200 else f"📝 Response: {message.content}")
        print("─" * 60)

async def run_sequential_orchestration(topic: str):
    """Demonstrates sequential Azure AI Agent orchestration using Semantic Kernel's SequentialOrchestration."""
    
    print(f"🔄 Starting Sequential Azure AI Agent Orchestration for: {topic}")
    print(f"{'='*80}")
    
    # Create our specialized Azure AI Agent team
    agents = await create_specialized_agents()
    
    # Create the sequential orchestration with our Azure AI Agents
    # The agents will be executed in the order they are provided
    orchestration = SequentialOrchestration(
        members=[agents['research'], agents['analytics'], agents['content']],
        agent_response_callback=agent_response_callback
    )
    
    # Create and start the runtime
    runtime = InProcessRuntime()
    runtime.start()
    
    try:
        print(f"\n🎯 Sequential Pipeline: Research → Analytics → Content")
        print(f"{'─'*60}")
        
        # Create the task that will flow through the pipeline
        task = f"""Please analyze the topic '{topic}' comprehensively:
        
        Research Agent: Gather comprehensive information, key facts, and recent developments about {topic}.
        Analytics Agent: Analyze the research data, identify trends, and create projections.
        Content Agent: Synthesize all findings into a well-formatted executive report.
        
        Each agent should build upon the previous agent's work to create a comprehensive analysis.
        Be very concise in your responses, focusing on key insights and actionable recommendations."""
        
        # Invoke the sequential orchestration
        orchestration_result = await orchestration.invoke(
            task=task,
            runtime=runtime
        )
        
        # Wait for the results
        final_result = await orchestration_result.get(timeout=1200)
        
        print(f"\n✅ Sequential Azure AI Agent Orchestration Complete!")
        print(f"🎯 Final Result: Comprehensive {topic} analysis through sequential pipeline")
        print(f"\n📄 Final Report:\n{final_result}")
        
        return final_result
        
    finally:
        # Stop the runtime
        await runtime.stop_when_idle()
        print(f"\n🧹 Sequential orchestration cleanup complete.")

print("✅ Sequential Azure AI Agent orchestration function ready!")

✅ Sequential Azure AI Agent orchestration function ready!


In [5]:
# Test Sequential Azure AI Agent Orchestration
sequential_topic = "Artificial Intelligence in Healthcare"

print(f"🚀 Testing Sequential Azure AI Agent Orchestration")
print(f"📋 Topic: {sequential_topic}")
print(f"⏱️ This will take a few moments as each Azure AI Agent works in sequence...")

start_time = time.time()
sequential_result = await run_sequential_orchestration(sequential_topic)
end_time = time.time()

print(f"\n⏱️ Sequential Azure AI Agent orchestration completed in {end_time - start_time:.2f} seconds")
print(f"\n🎉 Sequential Azure AI Agent Orchestration Success!")
print(f"📄 Generated comprehensive analysis with research, analytics, and professional formatting")

🚀 Testing Sequential Azure AI Agent Orchestration
📋 Topic: Artificial Intelligence in Healthcare
⏱️ This will take a few moments as each Azure AI Agent works in sequence...
🔄 Starting Sequential Azure AI Agent Orchestration for: Artificial Intelligence in Healthcare
🤖 Creating specialized Azure AI Agent team...
🔍 Creating Research Agent...
✅ Dr. Research: asst_FrdwyLLsdpfuFd5NXcolSpSn
📊 Creating Analytics Agent...
✅ Prof. Analytics: asst_SXW86gJS2EfXfSbT5SHOMNkv
📝 Creating Content Agent...
✅ Ms. Content: asst_l2KJhKQPbUgomGVuCUuC4duP
🎭 Specialized Azure AI Agent team created!
🔍 Dr. Research: Information gathering specialist
📊 Prof. Analytics: Data analysis expert
📝 Ms. Content: Communication specialist

🎯 Sequential Pipeline: Research → Analytics → Content
────────────────────────────────────────────────────────────
🔍 Research completed for: Artificial Intelligence in Healthcare

[12:21:00] 🤖 Azure AI Agent: dr_research
📝 Response: 
─────────────────────────────────────────────────────

## ⚡ Concurrent Orchestration: Parallel Pattern

In concurrent orchestration, multiple Azure AI Agents work simultaneously:

**Research Agent** ⟷ **Analytics Agent** ⟷ **Content Agent** → **Combine Results**

This pattern is perfect when:
- Azure AI Agents can work independently on different aspects
- Speed is important (parallel processing)
- Tasks don't depend on each other's outputs

Let's implement concurrent orchestration for rapid multi-perspective analysis!

In [6]:
async def run_concurrent_orchestration(topic: str):
    """Demonstrates concurrent Azure AI Agent orchestration using Semantic Kernel's ConcurrentOrchestration."""
    
    print(f"⚡ Starting Concurrent Azure AI Agent Orchestration for: {topic}")
    print(f"{'='*80}")
    
    # Create our specialized Azure AI Agent team
    agents = await create_specialized_agents()
    
    # Create the concurrent orchestration with our Azure AI Agents
    # All agents will work on the same task simultaneously
    orchestration = ConcurrentOrchestration(
        members=[agents['research'], agents['analytics'], agents['content']]
    )
    
    # Create and start the runtime
    runtime = InProcessRuntime()
    runtime.start()
    
    try:
        print(f"\n🎯 All Azure AI Agents working simultaneously on: {topic}")
        print(f"{'─'*60}")
        
        # Create the task for all agents to work on concurrently
        task = f"""Analyze '{topic}' from your specialized perspective:
        
        Research Agent: Focus on current trends, recent innovations, and market dynamics in {topic}.
        Analytics Agent: Analyze market data, growth patterns, and create projections for {topic}.
        Content Agent: Create an engaging framework and structure for a {topic} report.
        
        Each agent should work independently and provide insights from your area of expertise.
        Be very concise in your responses, focusing on key insights and actionable recommendations."""
        
        # Invoke the concurrent orchestration
        orchestration_result = await orchestration.invoke(
            task=task,
            runtime=runtime
        )
        
        # Wait for all agents to complete their tasks
        results = await orchestration_result.get(timeout=1200)
        
        print(f"\n🎉 All Azure AI Agents completed their tasks!")
        print(f"{'─'*60}")
        
        # Display individual results
        print(f"\n📊 Concurrent Analysis Results:")
        for i, result in enumerate(results, 1):
            result_preview = str(result.content)[:200] + "..." if len(str(result.content)) > 200 else str(result.content)
            print(f"\n   🤖 {result.name}:")
            print(f"   📝 {result_preview}")
        
        # Combine results for final output
        combined_result = f"""
# Concurrent Analysis Results for {topic}

## Multi-Agent Parallel Analysis
Generated by concurrent orchestration of specialized Azure AI Agents

{chr(10).join([f"### {result.name} Insights:{chr(10)}{result.content}{chr(10)}" for result in results])}

---
*Generated through concurrent Azure AI Agent orchestration using Semantic Kernel*
"""
        
        print(f"\n✅ Concurrent Azure AI Agent Orchestration Complete!")
        print(f"🎯 Final Result: Multi-perspective {topic} analysis from parallel Azure AI Agent work")
        
        return combined_result
        
    finally:
        # Stop the runtime
        await runtime.stop_when_idle()
        print(f"\n🧹 Concurrent orchestration cleanup complete.")

print("✅ Concurrent Azure AI Agent orchestration function ready!")

✅ Concurrent Azure AI Agent orchestration function ready!


In [7]:
# Test Concurrent Azure AI Agent Orchestration
concurrent_topic = "Renewable Energy Technologies"

print(f"⚡ Testing Concurrent Azure AI Agent Orchestration")
print(f"📋 Topic: {concurrent_topic}")
print(f"🚀 All Azure AI Agents will work simultaneously for maximum speed!")

start_time = time.time()
concurrent_result = await run_concurrent_orchestration(concurrent_topic)
end_time = time.time()

print(f"\n⏱️ Concurrent Azure AI Agent orchestration completed in {end_time - start_time:.2f} seconds")
print(f"\n🎉 Concurrent Azure AI Agent Orchestration Success!")
print(f"📄 Generated multi-perspective analysis through parallel Azure AI Agent collaboration")

⚡ Testing Concurrent Azure AI Agent Orchestration
📋 Topic: Renewable Energy Technologies
🚀 All Azure AI Agents will work simultaneously for maximum speed!
⚡ Starting Concurrent Azure AI Agent Orchestration for: Renewable Energy Technologies
🤖 Creating specialized Azure AI Agent team...
🔍 Creating Research Agent...
✅ Dr. Research: asst_6naVnKwyr9cgHVpxDUswjmTt
📊 Creating Analytics Agent...
✅ Prof. Analytics: asst_SauSqXQkdInls80Y47GnBg9d
📝 Creating Content Agent...
✅ Ms. Content: asst_lYnRUJXT1C1KwwxXvCmIIPnV
🎭 Specialized Azure AI Agent team created!
🔍 Dr. Research: Information gathering specialist
📊 Prof. Analytics: Data analysis expert
📝 Ms. Content: Communication specialist

🎯 All Azure AI Agents working simultaneously on: Renewable Energy Technologies
────────────────────────────────────────────────────────────
🔍 Research completed for: Current trends, recent innovations, and market dynamics in Renewable Energy Technologies
🔍 Research completed for: Market data, growth patterns, an

## 🎯 Real-World Azure AI Agent Orchestration Scenarios

Let's explore practical scenarios where you'd use each orchestration pattern with Azure AI Agents:

### 🔄 Sequential Azure AI Agent Orchestration Use Cases:

1. **Document Processing Pipeline**
   - OCR Azure AI Agent → Text Analysis Azure AI Agent → Summary Azure AI Agent
   
2. **Financial Analysis Workflow**
   - Data Collection Azure AI Agent → Risk Analysis Azure AI Agent → Report Generation Azure AI Agent
   
3. **Software Development Pipeline**
   - Code Analysis Azure AI Agent → Testing Azure AI Agent → Documentation Azure AI Agent

### ⚡ Concurrent Azure AI Agent Orchestration Use Cases:

1. **Multi-Source Research**
   - News Azure AI Agent + Academic Azure AI Agent + Social Media Azure AI Agent → Combine Insights
   
2. **Customer Service Routing**
   - Intent Azure AI Agent + Sentiment Azure AI Agent + Priority Azure AI Agent → Route to Human
   
3. **Content Generation**
   - Text Azure AI Agent + Image Azure AI Agent + SEO Azure AI Agent → Multi-media Content


## 🏆 Best Practices for Azure AI Agent Orchestration

### ✅ Design Principles:

1. **Azure AI Agent Specialization**
   - Give each Azure AI Agent a clear, focused role
   - Create specialized plugins for domain expertise
   - Write specific instructions for each Azure AI Agent

2. **Error Handling**
   - Always implement proper cleanup (Azure AI Agents are managed)
   - Handle agent failures gracefully
   - Provide fallback strategies

3. **Performance Optimization**
   - Use concurrent patterns for independent tasks
   - Use sequential patterns for dependent workflows
   - Monitor execution times and optimize bottlenecks

4. **Resource Management**
   - Azure AI Agents are managed by Azure (no manual cleanup needed)
   - Monitor Azure AI service usage and costs
   - Implement timeouts for long-running operations

### 🔒 Security Considerations:

1. **Data Privacy**
   - Don't log sensitive information
   - Implement data retention policies
   - Use secure communication between Azure AI Agents

2. **Access Control**
   - Use DefaultAzureCredential for secure authentication
   - Limit Azure AI Agent permissions appropriately
   - Validate all inputs to plugins
   - Monitor Azure AI Agent activities

### 📊 Monitoring and Debugging:

1. **Logging**
   - Log Azure AI Agent interactions and decisions
   - Track execution times and performance metrics
   - Monitor plugin function calls

2. **Testing**
   - Test individual Azure AI Agents before orchestration
   - Test orchestration patterns with various scenarios
   - Implement automated testing for critical workflows

## 🚀 Advanced Semantic Kernel Orchestration Features

Semantic Kernel provides powerful additional features for orchestration beyond basic sequential and concurrent patterns:

### 🔧 Advanced Capabilities:

1. **Cancellation Support** 🛑
   - Cancel orchestrations that are taking too long
   - Graceful handling of cancelled operations
   - Timeout management

2. **Structured Outputs** 📋  
   - Get results in predefined data structures
   - Type-safe orchestration results
   - JSON schema validation

3. **Agent Response Callbacks** 👂
   - Monitor agent interactions in real-time
   - Log and debug orchestration flows
   - Custom response processing

4. **Runtime Management** ⚙️
   - InProcessRuntime for local orchestration
   - Efficient resource management
   - Automatic cleanup

Let's explore these advanced features!

In [8]:
async def demonstrate_advanced_features():
    """Demonstrate advanced Semantic Kernel orchestration features with Azure AI Agents."""
    
    print("🚀 Advanced Azure AI Agent Orchestration Features Demo")
    print("="*50)
    
    # Create Azure AI Agents for demonstration
    agents = await create_specialized_agents()
    
    # 1. Demonstrate Agent Response Callback with Enhanced Monitoring
    print("\n👂 1. Enhanced Azure AI Agent Response Monitoring Demo")
    print("-" * 40)
    
    def detailed_callback(message: ChatMessageContent) -> None:
        """Enhanced callback with comprehensive details."""
        if message.role != AuthorRole.TOOL:
            timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]  # Include milliseconds
            print(f"\n[{timestamp}] 🤖 Azure AI Agent Response:")
            print(f"    📛 Name: {message.name}")
            print(f"    📝 Content Length: {len(message.content)} characters")
            print(f"    🔤 Content Preview: {message.content[:150]}{'...' if len(str(message.content)) > 150 else ''}")
            if hasattr(message, 'metadata') and message.metadata:
                print(f"    📊 Metadata: {message.metadata}")
            print("    " + "─" * 50)
    
    # Create a sequential orchestration with detailed monitoring
    monitored_orchestration = SequentialOrchestration(
        members=[agents['research']],  # Just one agent for demo
        agent_response_callback=detailed_callback
    )
    
    runtime = InProcessRuntime()
    runtime.start()
    
    try:
        print("🎯 Running monitored Azure AI Agent orchestration...")
        result = await monitored_orchestration.invoke(
            task="Provide a brief overview of renewable energy trends focusing on solar power adoption",
            runtime=runtime
        )
        
        final_result = await result.get(timeout=1200)
        print(f"\n✅ Monitored Azure AI Agent orchestration completed!")
        print(f"📄 Final Result Length: {len(str(final_result))} characters")
        
    except Exception as e:
        print(f"❌ Error in monitored orchestration: {e}")
        
    finally:
        await runtime.stop_when_idle()
    
    # 2. Demonstrate Timeout and Error Handling
    print("\n⏰ 2. Timeout and Error Handling Demo")
    print("-" * 40)
    
    quick_orchestration = SequentialOrchestration(
        members=[agents['analytics'], agents['content']],
        agent_response_callback=agent_response_callback
    )
    
    runtime2 = InProcessRuntime()
    runtime2.start()
    
    try:
        print("🎯 Testing timeout handling (using very short timeout for demo)...")
        result = await quick_orchestration.invoke(
            task="Analyze quantum computing market potential and create executive summary",
            runtime=runtime2
        )
        
        # Use a very short timeout to demonstrate timeout handling
        try:
            final_result = await result.get(timeout=2)  # Very short timeout for demo
            print(f"✅ Completed within timeout!")
        except asyncio.TimeoutError:
            print(f"⏰ Operation timed out as expected (demo timeout was very short)")
        except Exception as e:
            print(f"❌ Other error: {type(e).__name__}: {e}")
            
    finally:
        await runtime2.stop_when_idle()
    
    # 3. Demonstrate Concurrent with Multiple Specialized Tasks
    print("\n⚡ 3. Advanced Concurrent Azure AI Agent Orchestration Demo")
    print("-" * 40)
    
    concurrent_orchestration = ConcurrentOrchestration(
        members=[agents['research'], agents['analytics'], agents['content']]
    )
    
    runtime3 = InProcessRuntime()
    runtime3.start()
    
    try:
        print("🎯 Running advanced concurrent Azure AI Agent orchestration...")
        
        advanced_task = """Analyze 'Artificial Intelligence in Education' with each Azure AI Agent focusing on different aspects:
        
        Research Agent: Focus on current AI educational tools, case studies, and adoption rates in schools/universities.
        Analytics Agent: Analyze the effectiveness data, ROI metrics, and performance improvements from AI in education.
        Content Agent: Create a framework for an implementation guide for educational institutions considering AI adoption.
        
        Provide comprehensive insights from your specialized perspective."""
        
        result = await concurrent_orchestration.invoke(
            task=advanced_task,
            runtime=runtime3
        )
        
        results = await result.get(timeout=1200)
        
        print(f"\n🎉 Advanced concurrent Azure AI Agent orchestration completed!")
        print(f"📊 Generated {len(str(results))} specialized responses")
        
        for i, response in enumerate(results, 1):
            print(f"\n   🤖 Azure AI Agent {i} ({response.name}):")
            preview = response.content[:100] + "..." if len(str(response.content)) > 100 else response.content
            print(f"   📝 {preview}")
            
    except Exception as e:
        print(f"❌ Error in advanced concurrent orchestration: {e}")
        
    finally:
        await runtime3.stop_when_idle()
    
    print("\n🎉 Advanced Azure AI Agent features demonstration complete!")
    print("💡 Key takeaways:")
    print("   ✅ Agent response callbacks provide real-time monitoring")
    print("   ✅ Proper timeout handling prevents hung operations")
    print("   ✅ Both sequential and concurrent patterns work with advanced features")
    print("   ✅ Semantic Kernel provides robust Azure AI Agent orchestration infrastructure")

print("✅ Advanced Azure AI Agent features demo function ready!")


await demonstrate_advanced_features()

✅ Advanced Azure AI Agent features demo function ready!
🚀 Advanced Azure AI Agent Orchestration Features Demo
🤖 Creating specialized Azure AI Agent team...
🔍 Creating Research Agent...
✅ Dr. Research: asst_sI2YrltOZT4ZqyoMJ64qPTdS
📊 Creating Analytics Agent...
✅ Prof. Analytics: asst_JToFTRSjPpOvGg1juBXt73LN
📝 Creating Content Agent...
✅ Ms. Content: asst_DTDQFayVxbhqtRYkSeDj0ajj
🎭 Specialized Azure AI Agent team created!
🔍 Dr. Research: Information gathering specialist
📊 Prof. Analytics: Data analysis expert
📝 Ms. Content: Communication specialist

👂 1. Enhanced Azure AI Agent Response Monitoring Demo
----------------------------------------
🎯 Running monitored Azure AI Agent orchestration...
🔍 Research completed for: renewable energy trends with a focus on solar power adoption

[12:35:42.671] 🤖 Azure AI Agent Response:
    📛 Name: dr_research
    📝 Content Length: 0 characters
    🔤 Content Preview: 
    📊 Metadata: {'thread_id': 'thread_PJAVUDFFkub3VKiMReLts3rS'}
    ───────────────